In [3]:
module type SLOVAR = sig
  type ('k, 'v) t

  val poisci : ('k, 'v) t -> 'k -> 'v option
  val prazen : ('k, 'v) t
  val dodaj : 'k -> 'v -> ('k, 'v) t -> ('k, 'v) t
end

module type SLOVAR =
  sig
    type ('k, 'v) t
    val poisci : ('k, 'v) t -> 'k -> 'v option
    val prazen : ('k, 'v) t
    val dodaj : 'k -> 'v -> ('k, 'v) t -> ('k, 'v) t
  end


In [4]:
module Slovar : SLOVAR = struct
  type ('k, 'v) t = Prazno | Sestavljeno of int * ('k, 'v) t * 'k * 'v * ('k, 'v) t

  let rec poisci sl k =
    match sl with
    | Prazno -> None
    | Sestavljeno (_, l, k', v, d) when k = k' -> Some v
    | Sestavljeno (_, l, k', _, d) when k < k' -> poisci l k
    | Sestavljeno (_, l, k', _, d) when k > k' -> poisci d k
    | _ -> assert false

  let prazen = Prazno

  let visina drevo =
    match drevo with
    | Prazno -> 0
    | Sestavljeno (h, _, _, _, _) -> h

  let sestavljeno (l, k, v, d) =
    Sestavljeno (1 + max (visina l) (visina d), l, k, v, d)

  let zavrti_levo = function
    | Sestavljeno (_, l, k, v, Sestavljeno (_, dl, k', v', dd)) ->
        sestavljeno (sestavljeno (l, k, v, dl), k', v', dd)
    | _ -> failwith "Tega drevesa ne morem zavrteti"

  let zavrti_desno = function
    | Sestavljeno (_, Sestavljeno (_, ll, k, v, ld), k', v', d) ->
        sestavljeno (ll, k, v, sestavljeno (ld, k', v', d))
    | _ -> failwith "Tega drevesa ne morem zavrteti"

  let razlika = function
    | Prazno -> 0
    | Sestavljeno (_, l, _, _, d) -> visina l - visina d

  let uravnotezi drevo =
    match drevo with
    | Sestavljeno (_, l, k, v, d) when razlika drevo = 2 && razlika l = 1 ->
        zavrti_desno drevo
    | Sestavljeno (_, l, k, v, d) when razlika drevo = 2 ->
        sestavljeno (zavrti_levo l, k, v, d) |> zavrti_desno
    | Sestavljeno (_, l, k, v, d) when razlika drevo = -2 && razlika d = -1 ->
        zavrti_levo drevo
    | Sestavljeno (_, l, k, v, d) when razlika drevo = -2 ->
        sestavljeno (l, k, v, zavrti_desno d) |> zavrti_levo
    | _ -> drevo

  let rec dodaj k v drevo =
    match drevo with
    | Prazno -> Sestavljeno (1, Prazno, k, v, Prazno)
    | Sestavljeno (h, l, k', v', d) ->
        if k < k' then sestavljeno (dodaj k v l, k', v', d) |> uravnotezi
        else if k > k' then
          sestavljeno (l, k', v', dodaj k v d) |> uravnotezi
        else
          sestavljeno (l, k', v, d)
end

module Slovar : SLOVAR


In [5]:
let rec stolpi =
  function
  | 0 -> 1
  | n when n < 0 -> 0
  | n -> stolpi (n - 1) + stolpi (n - 2) + stolpi (n - 3)

val stolpi : int -> int = <fun>


In [7]:
stolpi 30

- : int = 53798080


In [6]:
let rec stolpi_s_slovarjem s n =
  match Slovar.poisci s n with
  | Some x -> x, s
  | None ->
    match n with
    | 0 -> 1, s
    | n when n < 0 -> 0, s
    | n ->
        let x1, s1 = stolpi_s_slovarjem s (n - 1) in
        let x2, s2 = stolpi_s_slovarjem s1 (n - 2) in
        let x3, s3 = stolpi_s_slovarjem s2 (n - 3) in
        let y = x1 + x2 + x3 in
        y, Slovar.dodaj n y s3

val stolpi_s_slovarjem :
  (int, int) Slovar.t -> int -> int * (int, int) Slovar.t = <fun>


In [8]:
stolpi_s_slovarjem Slovar.prazen 30

- : int * (int, int) Slovar.t = (53798080, <abstr>)


In [16]:
let slovar_za_stolpe = ref Slovar.prazen 

let rec stolpi_z_referenco n =
  match Slovar.poisci !slovar_za_stolpe n with
  | Some x -> x
  | None ->
    print_endline ("Računam za " ^ string_of_int n);
    match n with
    | 0 -> 1
    | n when n < 0 -> 0
    | n ->
        let y = stolpi_z_referenco (n - 1) + stolpi_z_referenco (n - 2) + stolpi_z_referenco (n - 3) in
        slovar_za_stolpe := Slovar.dodaj n y !slovar_za_stolpe;
        y

val slovar_za_stolpe : ('_weak3, '_weak4) Slovar.t ref = {contents = <abstr>}


val stolpi_z_referenco : int -> int = <fun>


In [22]:
let r = ref Slovar.prazen

val r : ('_weak11, '_weak12) Slovar.t ref = {contents = <abstr>}


In [25]:
r

- : (int, '_weak13) Slovar.t ref = {contents = <abstr>}


In [18]:
stolpi_z_referenco 30

- : int = 53798080


In [21]:
let razprsilna_tabela = Hashtbl.create 42

let rec stolpi_z_zgoscevalno_tabelo n =
  match Hashtbl.find_opt razprsilna_tabela n with
  | Some x -> x
  | None ->
    print_endline ("Računam za " ^ string_of_int n);
    match n with
    | 0 -> 1
    | n when n < 0 -> 0
    | n ->
        let y = stolpi_z_zgoscevalno_tabelo (n - 1) + stolpi_z_zgoscevalno_tabelo (n - 2) + stolpi_z_referenco (n - 3) in
        Hashtbl.add razprsilna_tabela n y;
        y

val razprsilna_tabela : ('_weak9, '_weak10) Hashtbl.t = <abstr>


val stolpi_z_zgoscevalno_tabelo : int -> int = <fun>


In [26]:
razprsilna_tabela

- : (int, int) Hashtbl.t = <abstr>


In [27]:
let id f = f

val id : 'a -> 'a = <fun>


In [35]:
let rec f x =
  g (x - 1)
and g = (fun x -> if x = 0 then 0 else f (x - 1))

val f : int -> int = <fun>
val g : int -> int = <fun>


In [33]:
let speed (f : ('a -> 'b)) : ('a -> 'b) = failwith "TODO"

val speed : ('a -> 'b) -> 'a -> 'b = <fun>


In [38]:
let slow_stolpi =
  function
  | 0 -> 1
  | n when n < 0 -> 0
  | n -> slow_stolpi (n - 1) + slow_stolpi (n - 2) + slow_stolpi (n - 3)


val slow_stolpi : (int -> int) -> int -> int = <fun>


In [39]:
let speed slow_f =
  let razprsilna_tabela = Hashtbl.create 42 in
  let rec speedy_f x =
    match Hashtbl.find_opt razprsilna_tabela x with
    | Some y -> y
    | None -> 
        let y = slow_f x in
        Hashtbl.add razprsilna_tabela x y;
        y
  in
  speedy_f

val speed : ('a -> 'b) -> 'a -> 'b = <fun>


In [41]:
let rec slow_stolpi =
  function
  | 0 -> 1
  | n when n < 0 -> 0
  | n -> slow_stolpi (n - 1) + slow_stolpi (n - 2) + slow_stolpi (n - 3)

val slow_stolpi : int -> int = <fun>


In [43]:
let speedy_stolpi = speed slow_stolpi

val speedy_stolpi : int -> int = <fun>


In [45]:
speedy_stolpi 30

- : int = 53798080


In [47]:
let rec slow_stolpi ena_druga_funkcija =
  function
  | 0 -> 1
  | n when n < 0 -> 0
  | n -> ena_druga_funkcija (n - 1) + ena_druga_funkcija (n - 2) + ena_druga_funkcija (n - 3)

let speed slow_f =
  let razprsilna_tabela = Hashtbl.create 42 in
  let rec speedy_f x =
    match Hashtbl.find_opt razprsilna_tabela x with
    | Some y -> y
    | None -> 
        let y = slow_f speedy_f x in
        Hashtbl.add razprsilna_tabela x y;
        y
  in
  speedy_f

let speedy_stolpi = speed slow_stolpi

val slow_stolpi : (int -> int) -> int -> int = <fun>


val speed : (('a -> 'b) -> 'a -> 'b) -> 'a -> 'b = <fun>


val speedy_stolpi : int -> int = <fun>


In [54]:
speedy_stolpi 600

- : int = -4024026665851594591


In [ ]:
let stolpi = speed @@ fun stolpi ->
  function
  | 0 -> 1
  | n when n < 0 -> 0
  | n -> stolpi (n - 1) + stolpi (n - 2) + stolpi (n - 3)

In [ ]:
let rec stolpi =
  function
  | 0 -> 1
  | n when n < 0 -> 0
  | n -> stolpi (n - 1) + stolpi (n - 2) + stolpi (n - 3)